# Análisis de Sentimientos con NLTK VADER
## Análisis de reseñas de productos de Amazon

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WillianReinaG/PNL_unidad1/blob/main/2_Analisis_Sentimientos_VADER.ipynb)

Este notebook realiza un análisis completo de sentimientos utilizando NLTK con VADER (Valence Aware Dictionary and sEntiment Reasoner).

## 1. Introducción a VADER

**VADER** es un lexicón y herramienta de análisis de sentimientos optimizada para redes sociales y textos cortos.

Características principales:
- Combina enfoques léxico y reglas basadas
- Excelente para textos informales
- Produce 4 scores: negativo, neutral, positivo y compound
- El **compound score** (-1 a +1) resume el sentimiento total

## 2. Instalación de dependencias

In [ ]:
!pip install nltk scikit-learn pandas numpy matplotlib seaborn -q

import nltk

nltk.download("vader_lexicon", quiet=True)
nltk.download("punkt", quiet=True)
nltk.download("averaged_perceptron_tagger", quiet=True)
nltk.download("wordnet", quiet=True)

print("✓ Dependencias instaladas")

## 3. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score,
    precision_score, recall_score, f1_score
)
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter

sia = SentimentIntensityAnalyzer()
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 5)

print("✓ Librerías importadas")

## 4. Cargar datos

In [ ]:
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !wget https://raw.githubusercontent.com/WillianReinaG/PNL_unidad1/main/amazon_cells_labelled.txt -O ./amazon_cells_labelled.txt

with open("./amazon_cells_labelled.txt", "r", encoding="utf-8") as file:
    df = pd.read_csv(file, sep="\t", header=None, names=["review", "sentiment"])

print(f"Datos cargados: {len(df)} reseñas")
print(f"Positivas: {(df['sentiment'] == 1).sum()} | Negativas: {(df['sentiment'] == 0).sum()}")
print("\nPrimeras 5 reseñas:") 
print(df.head())

## 5. Exploración de datos

In [ ]:
print("Información del dataset:")
print("-" * 50)
print(f"Total de reseñas: {len(df)}")
print(f"Caracteres promedio por reseña: {df['review'].str.len().mean():.0f}")
print(f"Palabras promedio por reseña: {df['review'].str.split().str.len().mean():.1f}")
print(f"\nDistribución de sentimientos:")
print(df['sentiment'].value_counts())
print(f"\nPorcentaje de cada clase:")
print(f"Positivas: {(df['sentiment'] == 1).sum() / len(df) * 100:.2f}%")
print(f"Negativas: {(df['sentiment'] == 0).sum() / len(df) * 100:.2f}%")

## 6. Limpieza de datos

In [ ]:
def clean_text(text):
    """Limpia el texto para análisis"""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s.!?,-]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

print("Limpiando textos...")
df['review_cleaned'] = df['review'].apply(clean_text)
df = df[df['review_cleaned'].str.len() > 0]

print(f"Reseñas después de limpieza: {len(df)}")
print("\nEjemplos de limpieza:")
for i in range(min(2, len(df))):
    print(f"Original: {df['review'].iloc[i][:60]}...")
    print(f"Limpia:   {df['review_cleaned'].iloc[i][:60]}...")
    print()

## 7. Ejemplo simple: Análisis manual de una reseña

In [ ]:
ejemplo_reseña = df['review_cleaned'].iloc[0]
scores = sia.polarity_scores(ejemplo_reseña)

print("ANÁLISIS DE UNA RESEÑA CON VADER")
print("=" * 60)
print(f"Reseña: {ejemplo_reseña}")
print("\nScores VADER:")
print(f"  Sentimiento Negativo: {scores['neg']:.4f}")
print(f"  Sentimiento Neutral:  {scores['neu']:.4f}")
print(f"  Sentimiento Positivo: {scores['pos']:.4f}")
print(f"  Compound Score:       {scores['compound']:.4f}")
print("\nInterpretación:")
if scores['compound'] >= 0.05:
    print("  → SENTIMIENTO POSITIVO")
elif scores['compound'] <= -0.05:
    print("  → SENTIMIENTO NEGATIVO")
else:
    print("  → SENTIMIENTO NEUTRAL")

## 8. Análisis VADER en todo el dataset

In [ ]:
print("Analizando {0} reseñas con VADER...".format(len(df)))
sentiment_scores = df['review_cleaned'].apply(lambda x: sia.polarity_scores(x))
sentiment_df = pd.DataFrame(sentiment_scores.tolist())

df = pd.concat([df, sentiment_df], axis=1)
df = df.rename(columns={
    "neg": "vader_negative",
    "neu": "vader_neutral",
    "pos": "vader_positive",
    "compound": "vader_compound"
})

print("✓ Análisis completado")
print("\nEstadísticas de Compound Scores:")
print(df['vader_compound'].describe())

## 9. Convertir scores a predicciones binarias

In [ ]:
df['vader_prediction'] = df['vader_compound'].apply(lambda x: 1 if x >= 0.05 else 0)

print("Predicciones generadas")
print("\nDistribución de predicciones:")
print(df['vader_prediction'].value_counts())
print("\nComparación predicciones vs reales:")
comparison = pd.crosstab(df['sentiment'], df['vader_prediction'], 
                         rownames=["Real"], colnames=["Predicción"])
print(comparison)

## 10. Cálculo de métricas de evaluación

In [ ]:
y_true = df['sentiment'].values
y_pred = df['vader_prediction'].values

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

print("=" * 70)
print("MÉTRICAS DE EVALUACIÓN")
print("=" * 70)
print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precision: {precision:.4f} (de positivas predichas, cuántas eran correctas)")
print(f"Recall:    {recall:.4f} (de positivas reales, cuántas detectó)")
print(f"F1-Score:  {f1:.4f} (media armónica de precision y recall)")

## 11. Matriz de confusión y reporte de clasificación

In [ ]:
cm = confusion_matrix(y_true, y_pred)

print("\nMatriz de Confusión:")
print("-" * 60)
print("{0:<20} {1:<20} {2:<20}".format("", "Pred. Negativo", "Pred. Positivo"))
print("{0:<20} {1:<20} {2:<20}".format("Real Negativo", str(cm[0,0]), str(cm[0,1])))
print("{0:<20} {1:<20} {2:<20}".format("Real Positivo", str(cm[1,0]), str(cm[1,1])))

print("\nReporte de Clasificación:")
print(classification_report(y_true, y_pred, target_names=['Negativo', 'Positivo']))

## 12. Visualización 1: Matriz de Confusión

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=["Negativo", "Positivo"],
            yticklabels=["Negativo", "Positivo"])
plt.title("Matriz de Confusión - VADER Sentiment Analysis")
plt.ylabel("Sentimiento Real")
plt.xlabel("Sentimiento Predicho")
plt.tight_layout()
plt.show()

## 13. Visualización 2: Distribución de Scores

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df['vader_compound'], bins=50, edgecolor="black", alpha=0.7)
axes[0].axvline(x=0.05, color="green", linestyle="--", label="Threshold Positivo")
axes[0].axvline(x=-0.05, color="red", linestyle="--", label="Threshold Negativo")
axes[0].set_xlabel("Compound Score")
axes[0].set_title("Distribución de Compound Scores")
axes[0].legend()

df.boxplot(column="vader_compound", by="sentiment", ax=axes[1])
axes[1].set_xlabel("Sentimiento Real (0=Neg, 1=Pos)")
axes[1].set_ylabel("Compound Score")
axes[1].set_title("Compound Por Sentimiento")
plt.suptitle("")
plt.tight_layout()
plt.show()

## 14. Análisis de componentes VADER

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].hist(df['vader_negative'], bins=30, alpha=0.7, edgecolor="black")
axes[0].set_title("Distribución de Negatividad")
axes[0].set_xlabel("Negative Score")

axes[1].hist(df['vader_neutral'], bins=30, alpha=0.7, edgecolor="black", color="orange")
axes[1].set_title("Distribución de Neutralidad")
axes[1].set_xlabel("Neutral Score")

axes[2].hist(df['vader_positive'], bins=30, alpha=0.7, edgecolor="black", color="green")
axes[2].set_title("Distribución de Positividad")
axes[2].set_xlabel("Positive Score")

plt.tight_layout()
plt.show()

## 15. Análisis de palabras más frecuentes

In [ ]:
positive_reviews = df[df['sentiment'] == 1]['review_cleaned'].str.cat(sep=' ')
negative_reviews = df[df['sentiment'] == 0]['review_cleaned'].str.cat(sep=' ')

positive_words = word_tokenize(positive_reviews)
negative_words = word_tokenize(negative_reviews)

positive_freq = Counter([w for w in positive_words if len(w) > 2 and w.isalpha()])
negative_freq = Counter([w for w in negative_words if len(w) > 2 and w.isalpha()])

print("Palabras más frecuentes en reseñas POSITIVAS:")
print("-" * 50)
for word, freq in positive_freq.most_common(15):
    print("{0:<15} {1:>5} veces".format(word, freq))

print("\n\nPalabras más frecuentes en reseñas NEGATIVAS:")
print("-" * 50)
for word, freq in negative_freq.most_common(15):
    print("{0:<15} {1:>5} veces".format(word, freq))

## 16. Análisis de errores (casos mal clasificados)

In [ ]:
df['correct'] = df['sentiment'] == df['vader_prediction']
misclassified = df[~df['correct']]

fp = misclassified[(misclassified['sentiment'] == 0) & (misclassified['vader_prediction'] == 1)]
fn = misclassified[(misclassified['sentiment'] == 1) & (misclassified['vader_prediction'] == 0)]

print("ANÁLISIS DE ERRORES")
print("=" * 70)
print(f"Correctas: {(df['correct']).sum()} ({(df['correct']).sum()/len(df)*100:.2f}%)")
print(f"Incorrectas: {len(misclassified)} ({len(misclassified)/len(df)*100:.2f}%)")
print(f"  - Falsos Positivos (real negativo, predicción positiva): {len(fp)}")
print(f"  - Falsos Negativos (real positivo, predicción negativa): {len(fn)}")

print("\n" + "=" * 70)
print("EJEMPLOS DE FALSOS POSITIVOS:")
print("=" * 70)
for idx in fp.head(3).index:
    print(f"Reseña: {df.loc[idx, 'review'][:70]}...")
    print(f"Score: {df.loc[idx, 'vader_compound']:.3f} (predicción: positivo, real: negativo)")
    print()

print("=" * 70)
print("EJEMPLOS DE FALSOS NEGATIVOS:")
print("=" * 70)
for idx in fn.head(3).index:
    print(f"Reseña: {df.loc[idx, 'review'][:70]}...")
    print(f"Score: {df.loc[idx, 'vader_compound']:.3f} (predicción: negativo, real: positivo)")
    print()

## 17. Resumen y conclusiones

In [ ]:
print("\n" + "=" * 70)
print("RESUMEN FINAL DEL ANÁLISIS")
print("=" * 70)
print(f"Dataset: {len(df)} reseñas")
print(f"Modelo: VADER (Valence Aware Dictionary and sEntiment Reasoner)")
print(f"\nRendimiento:")
print(f"  Accuracy:  {accuracy*100:.2f}%")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1-Score:  {f1:.4f}")
print(f"\nVentajas de VADER:")
print("  ✓ Excelente para textos cortos (redes sociales)")
print("  ✓ Maneja emojis y puntuación")
print("  ✓ Rápido y sin necesidad de entrenamiento")
print("  ✓ Interpretable")
print(f"\nLimitaciones:")
print("  ✗ Puede no captar sarcasmo bien")
print("  ✗ Sesgo hacia vocabulario común")
print("  ✗ Peor desempeño con textos largos")